In [1]:
import os
import re
import subprocess

def update_testbench_content(content, unit_name):
    # Define the content to be added
    additional_content = f'''
from cocotb_test.simulator import run
import logging
import pytest
log = logging.getLogger('cocotb_log_{unit_name}')
log.setLevel(logging.DEBUG)
'''

    # Find the last import line index
    last_import_index = None
    for i, line in enumerate(content):
        if 'import' in line or 'from' in line:
            last_import_index = i

    # Insert the additional content after the last import
    if last_import_index is not None:
        content.insert(last_import_index + 1, additional_content)

    # Replace print statements with log.info
    content = [re.sub(r'print\((.*?)\)', r'log.info(\1)', line) for line in content]

    return content

# Get the repository root directory
repo_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel']).strip().decode('utf-8')

# Define the path to your testbench files
testbench_dir = os.path.join(repo_root, 'val/common_level0')

# Iterate over each unit's testbench.py file
for unit in os.listdir(testbench_dir):
    unit_path = os.path.join(testbench_dir, unit)
    testbench_file_path = os.path.join(unit_path, 'testbench.py')
    
    # Only proceed if the testbench.py file exists
    if os.path.isfile(testbench_file_path):
        # Read the testbench file content
        with open(testbench_file_path, 'r') as file:
            content = file.readlines()

        # Update the content
        updated_content = update_testbench_content(content, unit)

        # Write the updated content back to the file
        with open(testbench_file_path, 'w') as file:
            file.writelines(updated_content)

print("Testbench files updated.")


Testbench files updated.
